In [ ]:
import kfp
from kfp.components import create_component_from_func
from kfp.dsl import pipeline


@create_component_from_func
def print_and_return_number(number: int) -> int:
    print(number)
    return number


@create_component_from_func
def sum_and_print_numbers(number_1: int, number_2: int):
    print(number_1 + number_2)


@pipeline(name="example_pipeline")
def example_pipeline(number_1: int, number_2: int):
    number_1_result = print_and_return_number(number_1).set_display_name("This is number 1")
    number_2_result = print_and_return_number(number_2).set_display_name("This is number 2")
    sum_result = sum_and_print_numbers(
        number_1=number_1_result.output, number_2=number_2_result.output
    ).set_display_name("This is sum of number 1 and number 2")

In [ ]:
import kfp
import requests

USERNAME = "user@example.com" # keycloak 사용시 불필요
PASSWORD = "12341234" # keycloak 사용시 불필요
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://istio-ingressgateway.istio-system/" # istio-ingressgateway pod ip:port ex) http://istio-ingressgateway.istio-system/ 

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}", # 크롬창 kubeflow dashboard 에서 관리자도구 -> storage -> cookies -> authservice_session 토큰 복붙
)
list_pipelines = client.list_pipelines()

print(list_pipelines.total_size)

#for i in range(list_pipelines.total_size):

In [ ]:
arguments={
    'number_1': 1,
    'number_2': 2
}

client.create_run_from_pipeline_func(pipeline_func=example_pipeline,arguments=arguments) # pipeline_func에 pipeline을 arguments에 인자값을 넣어주면 된다.